# LF JSON generator

Given a CSV file with LF parameters, generate JSON

In [167]:
import csv

In [168]:
csvfiles = [{'file':'lf_model_params_Karlsson1996.csv', 'source':'Karlsson 1996', 
             'header_lines': [0,1], 'header_titles':['gender','mode'], 'main_header': None,
             'index_cols': [0,1,2], 'index_titles': ['parameter','unit','formula'], 'main_index': 0},
            {'file':'lf_model_params_Kroger2009.csv', 'source':'Kroger 2009', 
             'header_lines': [0,1], 'header_titles':['gender','mode'], 'main_header': None,
             'index_cols': [0,1,2], 'index_titles': ['parameter','unit','formula'], 'main_index': 0},]

In [169]:

def old_lf_csv_to_json(csvfile):
    with open(csvfile,'r') as f:
        for lno, l in enumerate(f.readlines()):
            if lno in header_lines:
                splitline = (l.strip().split(sep))
                headers[header_titles[lno]] = [ss for ii,ss in enumerate(splitline) if ii not in index_cols]
            else:
                splitline = (l.strip().split(sep))
                if len(index_cols)>1:
                    idx = tuple(ss for ii,ss in enumerate(splitline) if ii in index_cols)
                else:
                    idx = splitline[index_cols[0]]
                data[idx] = [ss for ii,ss in enumerate(splitline) if ii not in index_cols]
                ncols = len(data[idx]) 
            nlines+=1

    if transpose:
        newheaders = {kk:[] for kk in index_titles}
        newdata = [{'headers':{}, 'values':{}} for ii in range(ncols)]

        if main_index is not None and len(index_cols)>1:
            indexdata = {}
            for key, c in data.items():
                kinfo = {}
                indexdata[key[main_index]] = kinfo
                for ii,(tk,k) in enumerate(zip(index_titles,key)):
                    if ii != main_index:
                        kinfo[tk] = k
        else:
            indexdata = []
            for key, c in data.items():
                kinfo = {}
                for ii,(tk,k) in enumerate(zip(index_titles,key)):
                    kinfo[tk] = k
            indexdata.append(kinfo)

        for key, c in data.items():
            if main_index is not None and len(index_cols)>1:
                k = key[0]
            else: 
                k = str(key)
            for htit, hval in headers.items():
                for nd, hvals, cellval in zip(newdata,hval,c):
                    nd['headers'][htit] = hvals
                    if cellval:
                        try:
                            nd['values'][k] = float(cellval)
                        except ValueError:
                            nd['values'][k] = cellval
                            
        data = newdata
        
    return {'keys': indexdata, 'table':data}


In [170]:

def lf_csv_to_json(csvfile, sep = ',', header_lines = [0], header_titles = ['phoneme'], main_header = None,
                   index_cols = [0], index_titles = ['segment nbr'], main_index = 0, transpose=True, num_to_arrays=True):

    headers = {}
    data = {}

    nlines = 0
    ncols = 0
    
    unnamed = 0

    with open(csvfile,'r') as f:
        for lno, l in enumerate(f.readlines()):
            if lno in header_lines:
                splitline = (l.strip().split(sep))
                headers[header_titles[lno]] = [ss for ii,ss in enumerate(splitline) if ii not in index_cols]
            else:
                splitline = (l.strip().split(sep))
                idx = []
                for ii,ss in enumerate(splitline):
                    if ii in index_cols:
                        if len(ss)==0:
                            ss = 'unnamed_'+str(unnamed)
                            unnamed+=1                            
                        idx.append(ss)
                if len(index_cols)>1:
                    idx = tuple(idx)
                else:
                    idx = idx[0]
                data[idx] = [ss for ii,ss in enumerate(splitline) if ii not in index_cols]
                ncols = len(data[idx]) 
            nlines+=1
            
    if transpose:
        newheaders = {kk:[] for kk in index_titles}
        newdata = [{'headers':{}, 'values':{}} for ii in range(ncols)]

        if main_index is not None and len(index_cols)>1:
            indexdata = {}
            for key, c in data.items():
                kinfo = {}
                indexdata[key[main_index]] = kinfo
                for ii,(tk,k) in enumerate(zip(index_titles,key)):
                    if ii != main_index:
                        kinfo[tk] = k
        else:
            indexdata = []
            for key, c in data.items():
                kinfo = {}
                for ii,(tk,k) in enumerate(zip(index_titles,key)):
                    kinfo[tk] = k
            indexdata.append(kinfo)

        for key, c in data.items():
            if main_index is not None and len(index_cols)>1:
                k = key[0]
            else: 
                k = str(key)
            for htit, hval in headers.items():
                for nd, hvals, cellval in zip(newdata,hval,c):
                    nd['headers'][htit] = hvals
                    if cellval:
                        try:
                            nd['values'][k] = float(cellval)
                        except ValueError:
                            nd['values'][k] = cellval
                            
                            
        data = newdata
        
    if num_to_arrays:
        for dd in data:
            nums = {}
            maxnum = 0
            array = []
            for key in dd['values']:
                try:
                    num = int(key)
                    nums[num] = dd['values'][key]
                    maxnum = max(maxnum,num)

                except ValueError:
                    pass
            array = [None]*(maxnum+1)
            for ii in nums:
                del(dd['values'][str(ii)])
                array[ii] = nums[ii]
            try:
                minnum = min(nums.keys())
            except ValueError:
                continue
            dd['values']['array']=array[minnum:]

        
    return {'keys': indexdata, 'table':data}




            

In [171]:
import re
import os

alljs = []

fargs = 'csvfile sep header_lines header_titles main_header index_cols index_titles main_index transpose'.split()

for rec in csvfiles:
    csvfile = os.path.join('../parameters/glottis',rec['file'])
    rec['csvfile'] = csvfile
    kwargs = {x:rec[x] for x  in set(rec.keys())&set(fargs)}
    otherkeys = {x:rec[x] for x  in set(rec.keys())-set(fargs)}
    
    js = lf_csv_to_json(**kwargs)
    for ln in js['table']:
        ln['headers'].update(otherkeys)
    alljs.append(js)

In [172]:
import jq
jexp = jq.compile('.[].table | .[]')
newjs=[x for x in jexp.input(alljs)]

In [174]:
import json
with open('../parameters/glottis/all_lf.json', 'w') as f:
    json.dump(newjs,f)

In [173]:
[x for x in jq.compile('.[].headers | with_entries(select([.key] | inside(["mode","gender","source"])))').input(newjs)]

[{'gender': 'Male', 'mode': 'normal', 'source': 'Karlsson 1996'},
 {'gender': 'Male', 'mode': 'low f0', 'source': 'Karlsson 1996'},
 {'gender': 'Male', 'mode': 'med f0', 'source': 'Karlsson 1996'},
 {'gender': 'Male', 'mode': 'high f0', 'source': 'Karlsson 1996'},
 {'gender': 'Male', 'mode': 'soft', 'source': 'Karlsson 1996'},
 {'gender': 'Male', 'mode': 'med', 'source': 'Karlsson 1996'},
 {'gender': 'Male', 'mode': 'loud', 'source': 'Karlsson 1996'},
 {'gender': 'Male', 'mode': 'breathy', 'source': 'Karlsson 1996'},
 {'gender': 'Male', 'mode': 'pressed', 'source': 'Karlsson 1996'},
 {'gender': 'Female', 'mode': 'normal', 'source': 'Karlsson 1996'},
 {'gender': 'Female', 'mode': 'low f0', 'source': 'Karlsson 1996'},
 {'gender': 'Female', 'mode': 'med f0', 'source': 'Karlsson 1996'},
 {'gender': 'Female', 'mode': 'high f0', 'source': 'Karlsson 1996'},
 {'gender': 'Female', 'mode': 'soft', 'source': 'Karlsson 1996'},
 {'gender': 'Female', 'mode': 'med', 'source': 'Karlsson 1996'},
 {'gen

In [140]:
js={'keys': indexdata, 'table':newdata}
js

{'keys': {'F0': {'unit': 'Hz', 'formula': '1/T0'},
  '1/2RG': {'unit': '%', 'formula': 'Tp/T0'},
  'FG': {'unit': 'Hz', 'formula': '1/2Tp'},
  'RE': {'unit': '%', 'formula': 'Te/T0'},
  'OQ': {'unit': '%', 'formula': 'Tc/T0'},
  'Ta': {'unit': 'ms', 'formula': ''},
  'Ra': {'unit': '%', 'formula': 'Ta/T0'},
  'Fa': {'unit': 'Hz', 'formula': '1/(2 pi Ta)'},
  'SP': {'unit': '%', 'formula': 'Tc/Tp -1'},
  'RK': {'unit': '%', 'formula': 'Te/Tp-1'},
  'flpl': {'unit': 'l/s', 'formula': ''},
  'fllk': {'unit': 'l/s', 'formula': ''},
  'flpl - fllk': {'unit': 'l/s', 'formula': ''}},
 'table': [{'headers': {'gender': 'Male', 'mode': 'Pressed'},
   'values': {'F0': 128.0,
    '1/2RG': 27.0,
    'FG': 229.0,
    'RE': 39.2,
    'OQ': 41.0,
    'Ta': 0.102,
    'Ra': 1.31,
    'Fa': 1620.0,
    'SP': 39.5,
    'RK': 39.5}},
  {'headers': {'gender': '', 'mode': 'Modal'},
   'values': {'F0': 126.0,
    '1/2RG': 38.0,
    'FG': 167.0,
    'RE': 52.0,
    'OQ': 54.0,
    'Ta': 0.16,
    'Ra': 2.02,


In [125]:
for dd in newdata:
    nums = {}
    maxnum = 0
    array = []
    for key in dd['values']:
        try:
            num = int(key)
            nums[num] = dd['values'][key]
            maxnum = max(maxnum,num)
            
        except ValueError:
            pass
    array = [None]*(maxnum+1)
    for ii in nums:
        del(dd['values'][str(ii)])
        array[ii] = nums[ii]
    minnum = min(nums.keys())
    dd['values']['array']=array[minnum:]
        

In [126]:
newdata

[{'headers': {'phoneme': 'i'},
  'values': {'delta': 0.396285,
   'length ': 16.67,
   'f1': 333.0,
   'f2': 2332.0,
   'f3': 2986.0,
   'array': [0.33,
    0.3,
    0.36,
    0.34,
    0.68,
    0.5,
    2.43,
    3.15,
    2.66,
    2.49,
    3.39,
    3.8,
    3.78,
    4.35,
    4.5,
    4.43,
    4.68,
    4.52,
    4.15,
    4.09,
    3.51,
    2.95,
    2.03,
    1.66,
    1.38,
    1.05,
    0.6,
    0.35,
    0.32,
    0.12,
    0.1,
    0.16,
    0.25,
    0.24,
    0.38,
    0.28,
    0.36,
    0.65,
    1.58,
    2.05,
    2.01,
    1.58]}},
 {'headers': {'phoneme': 'I'},
  'values': {'delta': 0.396285,
   'length ': 16.67,
   'f1': 518.0,
   'f2': 2004.0,
   'f3': 2605.0,
   'array': [0.2,
    0.17,
    0.18,
    0.18,
    0.1,
    1.08,
    1.66,
    1.64,
    1.19,
    0.92,
    1.13,
    2.48,
    2.76,
    2.97,
    3.43,
    3.32,
    3.48,
    3.96,
    3.79,
    3.88,
    3.47,
    2.98,
    2.62,
    2.37,
    1.99,
    1.9,
    1.7,
    1.44,
    1.45,
    1.06,
 